# LLM Queries with Knowledge Base Integration

In [ ]:
import os
import dotenv
import sys

dotenv.load_dotenv(".env")
sys.path.append(os.path.join(os.getcwd(), "..", "src"))

### Approach 1: Custom Client

This approach will use the `RetrievalAugmentedGenerationClient` class defined in `src/rag/utilities.py`. This will NOT require a Microsoft managed private endpoint for private access.

In [ ]:
from rag.utilities import RetrievalAugmentedGenerationClient

# Create orchestration client
rag_client = RetrievalAugmentedGenerationClient(
    open_ai_endpoint=os.getenv("AZURE_OPENAI_API_BASE"),
    open_ai_chat_deployment=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT"),
    open_ai_embedding_deployment=os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT"),
    search_endpoint=os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
    search_index_name=os.getenv("AZURE_AI_SEARCH_INDEX_NAME"),
    system_prompt_configuration_file="../src/rag/configuration.yaml"
)

In [ ]:
message_history = []
message_history = rag_client.get_answer("Which tent is the most waterproof?", message_history=message_history)

In [ ]:
for message in message_history:
    content = message['content'].split("Sources:")[0].strip()
    print(f"{message['role'].title()}: {content}\n")

In [ ]:
message_history = rag_client.get_answer("Tell me more about the Alpine Explorer Tent?", message_history=message_history)

In [ ]:
for message in message_history:
    content = message['content'].split("Sources:")[0].strip()
    print(f"{message['role'].title()}: {content}\n")

### Approach 2: Azure OpenAI Service REST API

This will require public access on Azure AI Search or a Microsoft managed private endpoint for private access.

In [ ]:
from azure.identity import DefaultAzureCredential
import requests

credential = DefaultAzureCredential()
access_token = credential.get_token("https://cognitiveservices.azure.com/.default")

open_ai_endpoint = os.getenv("AZURE_OPENAI_API_BASE")
open_ai_chat_deployment = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT")
open_ai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")

chat_endpoint = f"{open_ai_endpoint}/openai/deployments/{open_ai_chat_deployment}/extensions/chat/completions?api-version={open_ai_api_version}"

request_headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {access_token.token}",
}

request_payload = {
    "dataSources": [
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": os.getenv("AZURE_AI_SEARCH_ENDPOINT"),
                "indexName": os.getenv("AZURE_AI_SEARCH_INDEX_NAME"),
                "queryType": "vectorSemanticHybrid",
                "embeddingDeploymentName": os.getenv(
                    "AZURE_OPENAI_EMBEDDING_DEPLOYMENT"
                ),
                "fieldsMapping": {"titleField": "title", "urlField": "path"},
            },
        }
    ],
    "messages": [{"role": "user", "content": "Which tent is the most waterproof?"}],
}

response = requests.post(
    chat_endpoint,
    headers=request_headers,
    json=request_payload,
)

print(response.json()["choices"][0]["message"]["content"])